## My N-Gram Model
Create a smart keyboard that predicts next word based on previous word(s).  

Using SKLearn CountVectorizer object to convert text to frequency counts.  
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [309]:
#meta 8/26/2018
#prev: based on preliminary work with 3 small corpa in predictNextWord_CountVectorizer.ipynb

#prev: using a small sample of 2 data sources: twitter and news
#    got unigrams and bigrams
#    no need to pre-compute the matrix unigram x unigram which would give us all probs of a next unigram, given prev word in a bigram
#    only calculating prob vector given a start token
#    expanded to trigrams - minimal code changes, works with trigrams out of the "box"
#previously thought only need to look for n-grams starting with prev_token => incorrect
#    need to compute a subset of rows:
#    for bigrams, only 1 row where first word = prev_token
#    for trigrams, 1+ rows where second word = prev_token
#    for n-grams, 1+ rows where word before last = prev_token

#    using equal, startwith and endswith give only partial results
#    need to find a better way to find all n-grams with prev word == prev_token.

#prev: use regex to find all n-grams with prev word == prev_token.

#here: 2-gram model with training wheels
#    figured out 2-gram model first, according to N-gram model instructions.
#    used regex to find all n-grams with prev word == prev_token.
#    wrote function after "training wheels"

#next: 2-gram model with no training wheels

In [310]:
import time
import numpy as np
import pandas as pd
import re #for regex and pattern matching
import matplotlib.pyplot as plt #for drawing plots
%matplotlib inline

#NLP libraries
from sklearn.feature_extraction.text import CountVectorizer

#not used
#from collections import Counter #for document-term counting


In [311]:
### start clock
start_time=time.time()

### 0. Load Data
Data source is 3 files

In [312]:
#get rid of punctuation
#  re explanation: 
#  refer to https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
#  replaces not (^) word characters or spaces with the empty string. 
#  Be careful though, the \w matches underscore too usually for example
# originally was
#   words_news = re.sub(r'[^\w\s]','',open('sampleData/en_US.news_small.txt').read().lower())
#get rid of numbers too
text_twitter_in = re.sub(r'[^\w\s]|[\d]','',open('sampleData/en_US.twitter_small.txt').read().lower())
text_news_in = re.sub(r'[^\w\s]|[\d]','',open('sampleData/en_US.news_small.txt').read().lower())
text_blogs_in = re.sub(r'[^\w\s]|[\d]','',open('sampleData/en_US.blogs_small.txt').read().lower())



In [313]:
#preview - notice \n markers of each new line
text_news_in


'he wasnt home alone apparently\nthe st louis plant had to close it would die of old age workers had been making cars there since the onset of mass automotive production in the s\nwsus plans quickly became a hot topic on local online sites though most people applauded plans for the new biomedical center many deplored the potential loss of the building\nthe alaimo group of mount holly was up for a contract last fall to evaluate and suggest improvements to trenton water works but campaign finance records released this week show the two employees donated a total of  to the political action committee pac partners for progress in early june partners for progress reported it gave more than  in both direct and inkind contributions to mayor tony mack in the two weeks leading up to his victory in the mayoral runoff election june \nand when its often difficult to predict a laws impact legislators should think twice before carrying any bill is it absolutely necessary is it an issue serious enough

In [314]:
#capture beg and end of line with special delimeters <s> </s> 
text_twitter = re.sub(r'\n','</s> <s> ',text_twitter_in)
text_twitter = ' '.join((' <s>', text_twitter,'</s>'))

text_news = re.sub(r'\n',' </s> <s> ', text_news_in)
text_news = ' '.join(('<s>', text_news,'</s>'))

text_blogs = re.sub(r'\n',' </s> <s> ', text_blogs_in)
text_blogs = ' '.join(('<s>', text_blogs,'</s>'))

In [315]:
#preview
text_twitter #class string
#text_news
#text_blogs

' <s> how are you btw thanks for the rt you gonna be in dc anytime soon love to see you been way way too long</s> <s> when you meet someone special youll know your heart will beat more rapidly and youll smile for no reason</s> <s> theyve decided its more fun if i dont</s> <s> so tired d played lazer tag  ran a lot d ughh going to sleep like in  minutes </s> <s> words from a complete stranger made my birthday even better </s> <s> first cubs game ever wrigley field is gorgeous this is perfect go cubs go</s> <s> i no i get another day off from skool due to the wonderful snow  and this wakes me updamn thing</s> <s> im coo jus at work hella tired r u ever in cali</s> <s> the new sundrop commercial hehe love at first sight</s> <s> we need to reconnect this week</s> <s> i always wonder how the guys on the auctions shows learned to talk so fast all i hear is djsosnekspqnslanskam</s> <s> dammnnnnn what a catch</s> <s> such a great picture the green shirt totally brings out your eyes</s> <s> des

In [316]:
#combine all words
text_all = text_twitter + ' ' + text_news + ' ' + text_blogs
text_all2 = []
text_all2.append(text_all)


In [317]:
#validate counts
print ('Twitter words', len(text_twitter))
print ('News words', len(text_news))
print ('News words', len(text_blogs))
print ('All words', len(text_all2[0]))

text_all[-100:]

Twitter words 1692
News words 1901
News words 7649
All words 11244


't than anything but after staring at it for a while and all of us cheering he started to dig in </s>'

### Start with Unigrams

In [318]:
#get frequency counts
vectorizer = CountVectorizer(token_pattern=r'(?u)[\<[\/]*]?\b\w+\b[\>*]?', ngram_range=(1, 1))
#print(type(vectorizer))
print('Successful regex for this vocab - unigrams with index')

vectorizer.fit(text_all2)
print(vectorizer.vocabulary_)

unigrams = list(vectorizer.vocabulary_)
unigrams[-10:]


Successful regex for this vocab - unigrams with index
{'<s>': 1, 'how': 349, 'are': 48, 'you': 884, 'btw': 109, 'thanks': 759, 'for': 284, 'the': 762, 'rt': 649, 'gonna': 309, 'be': 73, 'in': 368, 'dc': 193, 'anytime': 41, 'soon': 706, 'love': 444, 'to': 782, 'see': 668, 'been': 79, 'way': 836, 'too': 787, 'long': 436, '</s>': 0, 'when': 848, 'meet': 469, 'someone': 705, 'special': 708, 'youll': 885, 'know': 406, 'your': 886, 'heart': 332, 'will': 857, 'beat': 75, 'more': 484, 'rapidly': 614, 'and': 34, 'smile': 701, 'no': 514, 'reason': 620, 'theyve': 770, 'decided': 196, 'its': 385, 'fun': 297, 'if': 355, 'i': 351, 'dont': 221, 'so': 703, 'tired': 781, 'd': 189, 'played': 578, 'lazer': 416, 'tag': 748, 'ran': 613, 'a': 2, 'lot': 442, 'ughh': 810, 'going': 308, 'sleep': 696, 'like': 430, 'minutes': 478, 'words': 866, 'from': 295, 'complete': 164, 'stranger': 726, 'made': 449, 'my': 496, 'birthday': 92, 'even': 248, 'better': 87, 'first': 278, 'cubs': 185, 'game': 298, 'ever': 250, 'wr

['face',
 'cake',
 'sister',
 'seemed',
 'scared',
 'staring',
 'us',
 'cheering',
 'started',
 'dig']

### Continue with Bigrams
Building unigram x unigram matrix results in bigram probabilities.  So need to generate frequency counts for bigrams.

In [319]:
#need unigrams and bigrams 
n=2
vectorizer = CountVectorizer(token_pattern=r'(?u)[\<[\/]*]?\b\w+\b[\>*]?', ngram_range=(1, n))
#dtm with 1 document
dtm = vectorizer.fit_transform(text_all2) #class 'scipy.sparse.csr.csr_matrix'
print ('DTM type ', type(dtm))

vocab_list = vectorizer.get_feature_names() #class list
vocab_total=len(vocab_list)
print('Successful vocab - with bigrams: ', vocab_total)

#preview vocab and verify trigrams
vocab_list[-20:] #class list

DTM type  <class 'scipy.sparse.csr.csr_matrix'>
Successful vocab - with bigrams:  2794


['you mr',
 'you need',
 'you out',
 'you produce',
 'you that',
 'you to',
 'you who',
 'you will',
 'youll',
 'youll know',
 'youll smile',
 'your',
 'your die',
 'your eyes',
 'your graduation',
 'your heart',
 'your mailbox',
 'your own',
 'yourself',
 'yourself about']

In [320]:
dtm

<1x2794 sparse matrix of type '<class 'numpy.int64'>'
	with 2794 stored elements in Compressed Sparse Row format>

In [321]:
#query dtm - only works with 1 row;
#if multiple rows, there's no instance of '</s> <s>'
print ('DTM type ', type(dtm))
ngram_value = '</s> <s>'
#ngram_value = 'am sam'
ngram_idx = vocab_list.index(ngram_value)
print ('Query dtm: how many times an n-gram occurs in the text')
dtm[0,ngram_idx]

DTM type  <class 'scipy.sparse.csr.csr_matrix'>
Query dtm: how many times an n-gram occurs in the text


55

#### From sparse matrix into NumPy array  
NumPy arrays supports a greater variety of operations than a list

In [322]:
#convert from current format, sparse matrix, into a normal numpy array 
print ('DTM type before: ', type(dtm))
dtm = dtm.toarray()
print ('DTM type after', type(dtm))
dtm[0]

DTM type before:  <class 'scipy.sparse.csr.csr_matrix'>
DTM type after <class 'numpy.ndarray'>


array([56, 55, 56, ...,  3,  1,  1], dtype=int64)

In [323]:
#convert python list storing vocab into numpy array
vocab = np.array(vocab_list)
vocab[:10]

array(['</s>', '</s> <s>', '<s>', '<s> a', '<s> although', '<s> and',
       '<s> april', '<s> beauty', '<s> but', '<s> chad'], dtype='<U25')

In [324]:
#query dtm
ngram_idx = list(vocab).index(ngram_value)
dtm[0,ngram_idx]

55

#### Using NumPy indexing is more natural

In [325]:
dtm[0,vocab == ngram_value]

array([55], dtype=int64)

#### Print frequency counts (aka dtm)

In [326]:
dtm[0,:]

array([56, 55, 56, ...,  3,  1,  1], dtype=int64)

In [327]:
#print dtm frequency counts
df = pd.DataFrame(dtm,columns = vocab)
df

,</s>,</s> <s>,<s>,<s> a,<s> although,<s> and,<s> april,<s> beauty,<s> but,<s> chad,...,youll smile,your,your die,your eyes,your graduation,your heart,your mailbox,your own,yourself,yourself about
0,56,55,56,1,1,1,1,1,1,1,...,1,8,1,1,1,1,1,3,1,1


### Calculate some bigram probabilities from this corpus - manually
Didn't think I could at this point.  Thought I had to build unigram to bigram matrix first.  

Wrong.  I have everything at this point.  Probably not efficient, but sufficient.  Worry about efficiency after figure out stats.


In [328]:
#P(I|<s>) = ?


In [329]:
#calculate some bigram probabilities - 
#P(I|<s>) = ? 

n_gram_of = '<s> i'
n_gram_given = '<s>'
p_bigram = dtm[0,list(vocab).index(n_gram_of)] / dtm[0,list(vocab).index(n_gram_given)]

p_bigram

0.07142857142857142

In [330]:
#P(get|I) = ?

n_gram_of = 'i get'
n_gram_given = 'i'
p_bigram = dtm[0,list(vocab).index(n_gram_of)] / dtm[0,list(vocab).index(n_gram_given)]

p_bigram

0.03571428571428571

In [331]:
#P(no|i)  = ?

n_gram_of = 'i no'
n_gram_given = 'no'
p_bigram = dtm[0,list(vocab).index(n_gram_of)] / dtm[0,list(vocab).index(n_gram_given)]

p_bigram 

0.14285714285714285

In [332]:
#P(<s>|</s>)  = .67

n_gram_of = '</s> <s>'
n_gram_given = '</s>'
p_bigram = dtm[0,list(vocab).index(n_gram_of)] / dtm[0,list(vocab).index(n_gram_given)]

p_bigram 

0.9821428571428571

### Calculate specific bigram probabilities - Loop vs Vectorized
Don't need to build a matrix of bigrams to unigram.  
Only need to compute a subset of rows:  
 - for bigrams, only 1 row where first word = prev token 

In [333]:
prev_token = 'your'
#Q: given 'your', what are all bigrams and their probabilites?
#P([bigrams]|your)  = [p1,p2,..., pn] 

#work with all vocab
#print(vocab)

#--check if vocab contains a unigram token and if yes, where
print ('vocab contains a unigram token? ', prev_token in vocab) #True
#vocab.index(prev_token) #works with list, doesn't work with numpy array
list(vocab).index(prev_token) #works with numpy array



vocab contains a unigram token?  True


2785

#### Find n-grams (bigrams n=2)
Use regex to find all n-grams with prev word == prev_token.

In [334]:
#build regex
my_regex = r'\b' + prev_token + r'\b \w+?$'
print (my_regex)

\byour\b \w+?$


In [335]:
#--check if vocab contains n-grams with prev word = prev_token 'your' (includes bigrams starting with prev_token)
for term in vocab:
    #print (term, prev_token)
    #print (term == prev_token)
    if re.search(my_regex, term):
        print (term)


your die
your eyes
your graduation
your heart
your mailbox
your own


#### Vectorized
no looping  

In [336]:
r = re.compile(my_regex)
eligible_ngrams_list = list(filter(r.search, vocab_list)) # Read Note
print(eligible_ngrams_list)

['your die', 'your eyes', 'your graduation', 'your heart', 'your mailbox', 'your own']


#### How to read it
Given previous word 'your', get all 'eligible' n-grams:  

Correct for bigrams: 
- Given previous word 'your', get all bigrams starting with prev_token.

Correct for trigrams:
- Given previus two words 'word_x your', get trigrams with prev_token as the word before last.  

Later for 3+ grams: 
- Given previus n words, get n-grams with prev_token as the word before last.  


Once we found all 'eligible' n-grams, compile a maxtrix with counts of relevant frequencies.

#### Compute prob of all bigrams 

Looping

In [337]:
unigram_count = dtm[0,vocab==prev_token]
print('Unigram ', prev_token, unigram_count)

for ngram in eligible_ngrams_list:
    print(ngram)
    #print(np.where(vocab==ngram, 1, -1))
    
    #index based
    ngram_idx = vocab_list.index(ngram)
    print('index ', ngram_idx)
    ngram_count2  = dtm[0,ngram_idx]
    print('count', ngram_count2)

    #less code - not index based
    #ngram_count  = dtm[0,vocab == ngram]
    #print('count', ngram_count)
    
    #compute prob of each bigram | prev_token
    ngram_prob = ngram_count2 / unigram_count
    print ('relative prob ', ngram_prob)

#eligible_idx = vocab_list.index([eligible_ngrams_list]) #$acerror value error - correct behavior
    

Unigram  your [8]
your die
index  2786
count 1
relative prob  [0.125]
your eyes
index  2787
count 1
relative prob  [0.125]
your graduation
index  2788
count 1
relative prob  [0.125]
your heart
index  2789
count 1
relative prob  [0.125]
your mailbox
index  2790
count 1
relative prob  [0.125]
your own
index  2791
count 3
relative prob  [0.375]


#### Vectorized 

Given prev_token, compute probability of all bigrams with first word == prev_token

In [338]:
#reminders: prev_token = 'your'
#  get a vector (np.array) where value == term
a = np.where(vocab==prev_token,1,0)
a.sum() #given our vocab can only be 1 (found) or 0 (not found)

#  get index of a n-gram
prev_token_idx = vocab_list.index(prev_token)
prev_token_idx

#  get all values from list where values match regex pattern
eligible_ngrams_list = list(filter(r.search, vocab_list))
eligible_ngrams_list
#note: this list gives us all bigrams we're interested in to build a 2-gram model

['your die',
 'your eyes',
 'your graduation',
 'your heart',
 'your mailbox',
 'your own']

### Next: get index of each bigram, compute relative probs and pick the bigram with the highest prob

In [339]:
# get index of each bigram
eligible_ngrams_idx = [i for i, w in enumerate(vocab_list) if re.search(my_regex,w)]
eligible_ngrams_idx


[2786, 2787, 2788, 2789, 2790, 2791]

In [340]:
# compute relative bigram probs 
#[print(i) for i, w in enumerate(vocab_list) if re.search(my_regex,w)]
eligible_ngrams_probs = [dtm[0,i]/unigram_count for i, w in enumerate(vocab_list) if re.search(my_regex,w)]
eligible_ngrams_probs


[array([0.125]),
 array([0.125]),
 array([0.125]),
 array([0.125]),
 array([0.125]),
 array([0.375])]

In [342]:
# pick the bigram with the highest prob - only one (if more than one, pick the first one)
best_ngram_idx = np.argmax(eligible_ngrams_probs)
print("Best ngram index: ", best_ngram_idx)
best_ngram_value = eligible_ngrams_list[best_ngram_idx]
print("Best ngram value: ",best_ngram_value)
next_word = best_ngram_value.split()[n-1]
print("Predict next word: ", next_word)

Best ngram index:  5
Best ngram value:  your own
Predict next word:  own


In [368]:
# [optional] - if want to consider more than one next predictions, and array has equal probs
# pick the bigram(s) with the highest prob - if more than one
best_ngrams_value = max(eligible_ngrams_probs)

#best_ngrams = np.isin(model_probs, best_ngrams_value)
#best_ngrams_idx = np.where(best_ngrams, 1, -1)
#best_ngrams_idx

In [431]:
#Predict next word - wrap into a function
#must have prev declared variables:
#vocab
#vocab_list
#dtm

def predictNextWord(prev_term):
    #reminders: prev_term = 'your'
    
    # get a vector (np.array) where value == prev_term
    prev_term_exists = np.where(vocab==prev_term,1,0)
    ##print("exists? ", prev_term_exists.sum())
    
    # if prev_term exists, predict next word
    if prev_term_exists.sum(): #given our vocab can only be 1 (found) or 0 (not found)

        ## get index of a n-gram
        ##prev_term_idx = vocab_list.index(prev_term)
        # get count of prev_term
        prev_term_count = dtm[0,vocab==prev_term]
        
        #build regex to find all n-grams with prev word == prev_term.
        this_regex = r'\b' + prev_term + r'\b \w+?$'
        ##print (this_regex)
        r = re.compile(this_regex)

        #  get all values from list where values match regex pattern
        eligible_ngrams_list = list(filter(r.search, vocab_list))
        print("Eligible ngram list: ", eligible_ngrams_list)
        
        # get index of each bigram
        eligible_ngrams_idx = [i for i, w in enumerate(vocab_list) if re.search(this_regex,w)]
        print("Eligible ngram index: ", eligible_ngrams_idx)
        
        # compute relative bigram probs 
        eligible_ngrams_probs = [dtm[0,i]/prev_term_count for i, w in enumerate(vocab_list) if re.search(this_regex,w)]
        print("Eligible ngram probs: ", eligible_ngrams_probs)
        
        # pick the bigram with the highest prob - only one (if more than one, pick the first one)
        best_ngram_idx = np.argmax(eligible_ngrams_probs)
        print("Best ngram index: ", best_ngram_idx)
        best_ngram_value = eligible_ngrams_list[best_ngram_idx]
        print("Best ngram value: ",best_ngram_value)
        next_word = best_ngram_value.split()[n-1]
        print("Predict next word: ", next_word)
        
        return next_word
    else:
        #if prev_term doesn't exist, deal with it later
        print ("No such term found")


### Test the function - Bigram only (with training wheels)
error handling:  
add if unigram exists but no bigrams

In [432]:
#check if such word exists
test_word ='with'
print ('Term exists? ', test_word in vocab_list)

predictNextWord(test_word)

Term exists?  True
Eligible ngram list:  ['with and', 'with but', 'with circle', 'with graduation', 'with great', 'with making', 'with my', 'with no', 'with not', 'with plots', 'with promise', 'with tes', 'with the', 'with your']
Eligible ngram index:  [2691, 2692, 2693, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704]
Eligible ngram probs:  [array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.11111111]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.05555556]), array([0.22222222]), array([0.05555556])]
Best ngram index:  12
Best ngram value:  with the
Predict next word:  the


'the'

#### Rough notes
discard later

In [213]:
#--check if vocab contains tokens starting with 'your' and if yes, where
#no loop, check if vocab contains bigram token(s) starting with prev_token 
#refer to https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.char.html
#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.core.defchararray.startswith.html#numpy.core.defchararray.startswith
np.core.defchararray.endswith(vocab, prev_token) #returns boolean array

#np.where(re.search(my_regex, vocab[:]),1,-1)
np.where(np.core.defchararray.startswith(vocab,prev_token),1,-1)


array([-1, -1, -1, ...,  1,  1,  1])

In [214]:
print('Found n-grams starting with: ', prev_token)
#bigrams_idx = np.where(np.core.defchararray.endswith(vocab, ' ' + prev_token))
bigrams_idx = np.where(np.core.defchararray.endswith(vocab, prev_token ))
bigrams_idx[0].tolist()
list(vocab[bigrams_idx])

Found n-grams starting with:  your


['by your',
 'in your',
 'know your',
 'only your',
 'out your',
 'through your',
 'watch your',
 'with your',
 'your']

#### Pausing here...
Pausing 3-gram model.  want to figure out 2-gram model first, according to N-gram model instructions.

From previoues notes: Need to find a better way to find all n-grams with prev word == prev_token.  
Then proceed to probability calculations...

The rest of this code is from the previous version... will resume once figure out the piece just mentioned.
#### How to read it
Given previous word 'your', get probabilities of the next word or the next two words.  Max probability wins.

Issue: not exactly what's needed.  Change in next take.

Correct for bigrams: 
- Given previous word 'your', get probabilities of the next word.

Want for trigrams:
- Given previus two words 'word_x your', get probabiliteis of the next word.  

In [215]:
#calc prob of relevant tokens
dtm[0, bigrams_idx[0].tolist()]
bigrams_prob = dtm[0, bigrams_idx[0].tolist()]/vocab_total

print("Bigrams and their probabilities: ")
print(vocab[bigrams_idx])
bigrams_prob.tolist()


Bigrams and their probabilities: 
['by your' 'in your' 'know your' 'only your' 'out your' 'through your'
 'watch your' 'with your' 'your']


[0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.00035790980672870435,
 0.002863278453829635]

### Xtra

#### Word Counts with CountVectorizer
https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [216]:
#$xtra - code snippet
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}
(1, 8)
<class 'scipy.sparse.csr.csr_matrix'>
[[1 1 1 1 1 1 1 2]]


In [217]:
#$xtra - regex example with a hard coded word
#--check if vocab contains bigrams starting with prev token 'your' 
for term in vocab:
    #print (term, prev_token)
    #print (term == prev_token)
    if re.search(r'^own \w+?$', term):
        print (term)
        
#--check if vocab contains n-grams with prev token 'your' except bigrams
print ('\n')
for term in vocab:
    #print (term, prev_token)
    #print (term == prev_token)
    if re.search(r' own \w+?$', term):
        #print (term.find(prev_token))
        print (term)

print ('\n--Together')        
#--(combine the two above) check if vocab contains n-grams with prev token 'your' including bigrams
for term in vocab:
    #print (term, prev_token)
    #print (term == prev_token)
    if re.search(r'\bown\b \w+?$', term):
        #print (term.find(prev_token))
        print (term)

own environment
own fairytale
own innermost



--Together
own environment
own fairytale
own innermost


In [218]:
#$xtra - vectorized lookup: regex example with a hard coded word; works with list.  gave up on making it work with np.array
r = re.compile(r'\bown\b \w+?$')
newlist = list(filter(r.search, vocab_list)) # Read Note
print(newlist)

['own environment', 'own fairytale', 'own innermost']
